# 🚀 Atheria 4: Entrenamiento Sin UI

Este notebook permite entrenar modelos de Atheria 4 directamente en Kaggle o Google Colab **sin necesidad de la UI web**.

## 📋 Características

- ✅ Entrenamiento completo desde cero o continuar desde checkpoint
- ✅ Soporte para todas las arquitecturas: UNet, SNN_UNET, MLP, DEEP_QCA, UNET_CONVLSTM
- ✅ Transfer Learning desde experimentos existentes
- ✅ Sistema Smart Save (retiene mejores checkpoints automáticamente)
- ✅ Visualización básica de métricas durante entrenamiento
- ✅ Exportación de modelos TorchScript para inferencia

## 🔧 Requisitos

- PyTorch (CUDA disponible en Kaggle/Colab)
- Dependencias del proyecto Atheria 4

## 📦 Paso 1: Instalación de Dependencias

Primero, instalamos las dependencias necesarias. Este paso detecta automáticamente si estamos en Kaggle o Colab.

In [ ]:
# Detectar entorno (Kaggle o Colab)
import os
IN_KAGGLE = os.path.exists('/kaggle/input')
try:
    IN_COLAB = 'google.colab' in str(get_ipython())
except:
    IN_COLAB = False

print(f"🌍 Entorno detectado: {'Kaggle' if IN_KAGGLE else 'Colab' if IN_COLAB else 'Local'}")

# Instalar dependencias básicas
!pip install -q snntorch scikit-learn

# Para Colab: instalar pybind11 si es necesario (opcional, solo si usas motor nativo)
if IN_COLAB:
    !pip install -q pybind11

print("✅ Dependencias instaladas")

## 📂 Paso 2: Configuración del Proyecto

Aquí configuramos las rutas del proyecto. En Kaggle/Colab, podemos clonar el repo o usar archivos ya subidos.

In [ ]:
import sys
from pathlib import Path
import os

# Configurar ruta del proyecto
if IN_KAGGLE:
    # En Kaggle, asumimos que el proyecto está en /kaggle/working
    PROJECT_ROOT = Path("/kaggle/working/Atheria")
    if not PROJECT_ROOT.exists():
        print("⚠️ Proyecto no encontrado. Clonando desde GitHub...")
        !git clone https://github.com/Jonakss/Atheria.git /kaggle/working/Atheria
elif IN_COLAB:
    # En Colab, clonamos el repo
    PROJECT_ROOT = Path("/content/Atheria")
    if not PROJECT_ROOT.exists():
        print("📥 Clonando proyecto desde GitHub...")
        !git clone https://github.com/Jonakss/Atheria.git /content/Atheria
else:
    # Local: usar ruta actual
    PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()

# Agregar al path de Python
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"📁 Proyecto configurado en: {PROJECT_ROOT}")

# Verificar estructura básica
src_path = PROJECT_ROOT / "src"
if src_path.exists():
    print("✅ Estructura del proyecto verificada")
else:
    print("❌ Error: No se encontró la carpeta 'src'. Verifica la instalación.")

## ⚙️ Paso 3: Configuración del Entrenamiento

Configura aquí los parámetros de tu experimento. Puedes crear múltiples configuraciones para diferentes experimentos.

In [ ]:
from types import SimpleNamespace
import torch

# ============================================================================
# CONFIGURACIÓN DEL EXPERIMENTO
# ============================================================================
# Modifica estos valores según tu experimento

EXPERIMENT_CONFIG = {
    # Nombre del experimento (debe ser único)
    "EXPERIMENT_NAME": "UNET_32ch_D8_Colab",
    
    # Arquitectura del modelo
    # Opciones: "UNET", "SNN_UNET", "MLP", "DEEP_QCA", "UNET_CONVLSTM", "UNET_UNITARY"
    "MODEL_ARCHITECTURE": "UNET",
    
    # Parámetros del modelo
    "MODEL_PARAMS": {
        "d_state": 8,           # Dimensión del estado cuántico
        "hidden_channels": 32,  # Canales ocultos (más = más capacidad, más lento)
        # Para SNN_UNET:
        # "alpha": 0.9,
        # "beta": 0.85,
    },
    
    # Parámetros de entrenamiento
    "GRID_SIZE_TRAINING": 64,     # Tamaño del grid (64, 128, 256...)
    "QCA_STEPS_TRAINING": 100,    # Pasos de simulación por episodio
    "LR_RATE_M": 1e-4,            # Learning rate (0.0001)
    "GAMMA_DECAY": 0.01,          # Término Lindbladian (decaimiento)
    
    # Configuración del entrenamiento
    "TOTAL_EPISODES": 500,        # Total de episodios a entrenar
    "SAVE_EVERY_EPISODES": 50,    # Guardar checkpoint cada N episodios
    "EPISODES_TO_ADD": 1,         # Episodios a agregar por lote (usar 1)
    "STEPS_PER_EPISODE": 100,     # Pasos por episodio (debe coincidir con QCA_STEPS_TRAINING)
    
    # Opcional: Transfer Learning
    # "LOAD_FROM_EXPERIMENT": None,  # Nombre del experimento base (None = desde cero)
    
    # Opcional: Continuar entrenamiento
    # "CONTINUE_TRAINING": False,     # True = continuar desde último checkpoint
}

# Convertir a SimpleNamespace para compatibilidad
exp_cfg = SimpleNamespace(**EXPERIMENT_CONFIG)
exp_cfg.MODEL_PARAMS = SimpleNamespace(**EXPERIMENT_CONFIG["MODEL_PARAMS"])

# Detectar dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
exp_cfg.DEVICE = device

print("=" * 70)
print("📊 CONFIGURACIÓN DEL EXPERIMENTO")
print("=" * 70)
print(f"Nombre: {exp_cfg.EXPERIMENT_NAME}")
print(f"Arquitectura: {exp_cfg.MODEL_ARCHITECTURE}")
print(f"Device: {device}")
print(f"Grid Size: {exp_cfg.GRID_SIZE_TRAINING}x{exp_cfg.GRID_SIZE_TRAINING}")
print(f"Total Episodios: {exp_cfg.TOTAL_EPISODES}")
print(f"Learning Rate: {exp_cfg.LR_RATE_M}")
print("=" * 70)

## 🚀 Paso 4: Iniciar Entrenamiento

Aquí ejecutamos el pipeline de entrenamiento. El sistema guardará automáticamente:
- **Mejores checkpoints** (hasta 5 por defecto)
- **Último checkpoint** (siempre)
- **Checkpoints periódicos** (cada `SAVE_EVERY_EPISODES`)
- **Log de métricas** en Markdown (`docs/40_Experiments/`)

In [ ]:
import logging
from src.pipelines.pipeline_train import run_training_pipeline
from src.utils import get_latest_checkpoint

# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Verificar si hay checkpoint previo (continuar entrenamiento)
checkpoint_path = None
if hasattr(exp_cfg, 'CONTINUE_TRAINING') and exp_cfg.CONTINUE_TRAINING:
    checkpoint_path = get_latest_checkpoint(exp_cfg.EXPERIMENT_NAME)
    if checkpoint_path:
        logger.info(f"✅ Checkpoint encontrado: {checkpoint_path}")
        logger.info("🔄 Continuando entrenamiento desde checkpoint...")
    else:
        logger.warning("⚠️ No se encontró checkpoint. Iniciando desde cero.")
        checkpoint_path = None

# Ejecutar entrenamiento
print("\n" + "=" * 70)
print("🎯 INICIANDO ENTRENAMIENTO")
print("=" * 70)

try:
    run_training_pipeline(exp_cfg, checkpoint_path=checkpoint_path)
    print("\n" + "=" * 70)
    print("✅ ENTRENAMIENTO COMPLETADO EXITOSAMENTE")
    print("=" * 70)
except Exception as e:
    print("\n" + "=" * 70)
    print(f"❌ ERROR DURANTE EL ENTRENAMIENTO: {e}")
    print("=" * 70)
    raise

## 📊 Paso 5: Visualizar Resultados

Visualiza las métricas de entrenamiento y el progreso del experimento.

In [ ]:
import json
from pathlib import Path

# Cargar configuración del experimento
experiment_dir = PROJECT_ROOT / "output" / "experiments" / exp_cfg.EXPERIMENT_NAME
config_path = experiment_dir / "config.json"

if config_path.exists():
    with open(config_path, 'r') as f:
        saved_config = json.load(f)
    
    print("📋 Configuración guardada del experimento:")
    print(json.dumps(saved_config, indent=2))
    
    # Cargar log de resultados si existe
    log_path = PROJECT_ROOT / "docs" / "40_Experiments" / f"{exp_cfg.EXPERIMENT_NAME}_LOG.md"
    if log_path.exists():
        print(f"\n📄 Log de resultados disponible en: {log_path}")
        with open(log_path, 'r') as f:
            log_content = f.read()
            # Mostrar últimas 50 líneas
            lines = log_content.split('\n')
            print("\n".join(lines[-50:]))
else:
    print("⚠️ No se encontró configuración guardada del experimento.")

In [ ]:
# Listar checkpoints guardados
from src.utils import get_latest_checkpoint
import os

checkpoint_dir = PROJECT_ROOT / "output" / "checkpoints" / exp_cfg.EXPERIMENT_NAME
if checkpoint_dir.exists():
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.endswith('.pth')]
    print(f"\n💾 Checkpoints guardados ({len(checkpoints)}):")
    for cp in sorted(checkpoints):
        cp_path = checkpoint_dir / cp
        size_mb = cp_path.stat().st_size / (1024 * 1024)
        print(f"  - {cp} ({size_mb:.2f} MB)")
    
    latest = get_latest_checkpoint(exp_cfg.EXPERIMENT_NAME)
    if latest:
        print(f"\n✅ Último checkpoint: {latest}")
else:
    print("⚠️ No se encontraron checkpoints.")

## 📦 Paso 6: Exportar Modelo TorchScript (Opcional)

Exporta el modelo entrenado a TorchScript para usar en inferencia o el motor nativo C++.

In [ ]:
# Exportar modelo a TorchScript para inferencia
from src.model_loader import load_model
import torch

checkpoint_path = get_latest_checkpoint(exp_cfg.EXPERIMENT_NAME)
if checkpoint_path:
    print(f"📤 Exportando modelo desde: {checkpoint_path}")
    
    # Cargar modelo
    model, state_dict = load_model(exp_cfg, checkpoint_path)
    if model:
        model.eval()
        model.to(device)
        
        # Crear ejemplo de input
        example_input = torch.randn(1, 2 * exp_cfg.MODEL_PARAMS.d_state, 
                                    exp_cfg.GRID_SIZE_TRAINING, 
                                    exp_cfg.GRID_SIZE_TRAINING, device=device)
        
        # Exportar a TorchScript
        torchscript_dir = PROJECT_ROOT / "output" / "torchscript_models" / exp_cfg.EXPERIMENT_NAME
        torchscript_dir.mkdir(parents=True, exist_ok=True)
        torchscript_path = torchscript_dir / "model.pt"
        
        try:
            traced_model = torch.jit.trace(model, example_input, strict=False)
            traced_model.save(str(torchscript_path))
            print(f"✅ Modelo exportado a: {torchscript_path}")
        except Exception as e:
            print(f"⚠️ Error exportando modelo: {e}")
            print("   Intentando con torch.jit.script...")
            try:
                scripted_model = torch.jit.script(model)
                scripted_model.save(str(torchscript_path))
                print(f"✅ Modelo exportado (script) a: {torchscript_path}")
            except Exception as e2:
                print(f"❌ Error exportando modelo: {e2}")
else:
    print("⚠️ No se encontró checkpoint para exportar.")

## 💾 Paso 7: Descargar Resultados (Colab/Kaggle)

Si estás en Colab o Kaggle, puedes descargar los resultados del entrenamiento.

In [ ]:
# En Colab: comprimir resultados para descarga
if IN_COLAB:
    import zipfile
    
    print("📦 Comprimiendo resultados...")
    zip_path = f"/content/{exp_cfg.EXPERIMENT_NAME}_results.zip"
    
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Agregar checkpoints
        checkpoint_dir = PROJECT_ROOT / "output" / "checkpoints" / exp_cfg.EXPERIMENT_NAME
        if checkpoint_dir.exists():
            for cp_file in checkpoint_dir.glob("*.pth"):
                zipf.write(cp_file, f"checkpoints/{cp_file.name}")
        
        # Agregar configuración
        config_file = PROJECT_ROOT / "output" / "experiments" / exp_cfg.EXPERIMENT_NAME / "config.json"
        if config_file.exists():
            zipf.write(config_file, f"experiments/{exp_cfg.EXPERIMENT_NAME}/config.json")
        
        # Agregar modelo TorchScript si existe
        torchscript_file = PROJECT_ROOT / "output" / "torchscript_models" / exp_cfg.EXPERIMENT_NAME / "model.pt"
        if torchscript_file.exists():
            zipf.write(torchscript_file, f"torchscript_models/{exp_cfg.EXPERIMENT_NAME}/model.pt")
    
    print(f"✅ Archivo comprimido creado: {zip_path}")
    print(f"📥 Descarga el archivo desde el panel de archivos de Colab")
    
elif IN_KAGGLE:
    print("📁 En Kaggle, los resultados están en /kaggle/working")
    print(f"   - Checkpoints: output/checkpoints/{exp_cfg.EXPERIMENT_NAME}/")
    print(f"   - Config: output/experiments/{exp_cfg.EXPERIMENT_NAME}/config.json")
else:
    print("💻 En entorno local, los resultados están en:")
    print(f"   - Checkpoints: output/checkpoints/{exp_cfg.EXPERIMENT_NAME}/")
    print(f"   - Config: output/experiments/{exp_cfg.EXPERIMENT_NAME}/config.json")

## 📝 Ejemplos de Configuración

Aquí hay algunos ejemplos de configuraciones para diferentes experimentos:

In [ ]:
# Ejemplo 1: U-Net pequeño (rápido, bueno para pruebas)
EXAMPLE_UNET_SMALL = {
    "EXPERIMENT_NAME": "UNET_16ch_D4_Test",
    "MODEL_ARCHITECTURE": "UNET",
    "MODEL_PARAMS": {"d_state": 4, "hidden_channels": 16},
    "GRID_SIZE_TRAINING": 32,
    "QCA_STEPS_TRAINING": 50,
    "LR_RATE_M": 1e-4,
    "TOTAL_EPISODES": 100,
}

# Ejemplo 2: U-Net mediano (balance)
EXAMPLE_UNET_MEDIUM = {
    "EXPERIMENT_NAME": "UNET_32ch_D8_Colab",
    "MODEL_ARCHITECTURE": "UNET",
    "MODEL_PARAMS": {"d_state": 8, "hidden_channels": 32},
    "GRID_SIZE_TRAINING": 64,
    "QCA_STEPS_TRAINING": 100,
    "LR_RATE_M": 1e-4,
    "TOTAL_EPISODES": 500,
}

# Ejemplo 3: SNN U-Net (Spiking Neural Network)
EXAMPLE_SNN_UNET = {
    "EXPERIMENT_NAME": "SNN_UNET_32ch_D8",
    "MODEL_ARCHITECTURE": "SNN_UNET",
    "MODEL_PARAMS": {"d_state": 8, "hidden_channels": 32, "alpha": 0.9, "beta": 0.85},
    "GRID_SIZE_TRAINING": 64,
    "QCA_STEPS_TRAINING": 100,
    "LR_RATE_M": 1e-4,
    "TOTAL_EPISODES": 500,
}

# Ejemplo 4: Transfer Learning (continuar desde otro experimento)
EXAMPLE_TRANSFER_LEARNING = {
    "EXPERIMENT_NAME": "UNET_64ch_D16_Transfer",
    "MODEL_ARCHITECTURE": "UNET",
    "MODEL_PARAMS": {"d_state": 16, "hidden_channels": 64},
    "GRID_SIZE_TRAINING": 128,
    "QCA_STEPS_TRAINING": 200,
    "LR_RATE_M": 5e-5,  # Learning rate más bajo para fine-tuning
    "TOTAL_EPISODES": 300,
    "LOAD_FROM_EXPERIMENT": "UNET_32ch_D8_Colab",  # Experimento base
}

print("💡 Ejemplos de configuración definidos.")
print("   Copia y pega uno de estos ejemplos en EXPERIMENT_CONFIG para usarlo.")